In [1]:
!pip install -U peft transformers accelerate datasets bitsandbytes -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.5.5 requires numpy<2.0.0,>=1.22.5, but you have numpy 2.2.5 which is incompatible.
langchain-chroma 0.1.2 requires numpy<2,>=1; python_version < "3.12", but you have numpy 2.2.5 which is incompatible.

[notice] A new release of pip is available: 24.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
from sklearn.metrics import accuracy_score, f1_score
import os
import logging


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\incha\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\incha\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\incha\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\incha\AppData\Roaming\Python\Python310\site-packages\traitlets\config\applicatio

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\incha\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\incha\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\incha\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\incha\AppData\Roaming\Python\Python310\site-packages\traitlets\config\applicatio

AttributeError: _UFUNC_API not found

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
numpy.core.umath failed to import

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
logger.info("Loading dataset")
dataset = load_dataset("hendzh/PromptShield")
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

MAX_LEN = 256
BATCH_SIZE = 16
EPOCHS = 3

In [ ]:
def tokenize_fn(example):
    return tokenizer(
        example["prompt"], 
        padding="max_length", 
        truncation=True, 
        max_length=MAX_LEN
    )

logger.info("Tokenizing dataset")
tokenized = dataset.map(tokenize_fn, batched=True)
train_dataset = tokenized["train"].with_format("torch")
val_dataset = tokenized["validation"].with_format("torch")

In [ ]:
from transformers import BitsAndBytesConfig

logger.info(f"Loading model: {model_name}")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

base_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    quantization_config=bnb_config,
    device_map="auto"
)


In [3]:

# ✅ Prepare model for QLoRA training
logger.info("Preparing model for QLoRA training")
base_model = prepare_model_for_kbit_training(base_model)

# Create a directory for logs and outputs
output_dir = "./qlora-bert-base"
os.makedirs(output_dir, exist_ok=True)

# ✅ Define QLoRA config
logger.info("Configuring LoRA")

# First try the full approach with module name patterns
try:
    lora_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        r=8,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        target_modules=["query", "key", "value"]  # Simple module name approach
    )
    model = get_peft_model(base_model, lora_config)
    logger.info("Successfully created PEFT model with 'query', 'key', 'value' target modules!")
except Exception as e:
    logger.warning(f"First approach failed: {e}")
    
    # Try a second approach with regex patterns
    try:
        lora_config = LoraConfig(
            task_type=TaskType.SEQ_CLS,
            r=8,
            lora_alpha=16,
            lora_dropout=0.1,
            bias="none",
            target_modules=[".*dense$"]  # Target dense modules with regex pattern
        )
        model = get_peft_model(base_model, lora_config)
        logger.info("Successfully created PEFT model with regex pattern target modules!")
    except Exception as e2:
        logger.warning(f"Second approach failed: {e2}")
        
        # Try a third approach with explicit modules
        logger.info("Trying explicit module approach...")
        # Find all linear layers in the model
        linear_modules = []
        for name, module in base_model.named_modules():
            if isinstance(module, torch.nn.Linear):
                linear_modules.append(name)
        
        logger.info(f"Found {len(linear_modules)} linear modules")
        if linear_modules:
            logger.info(f"Examples: {linear_modules[:3]}")
            
            lora_config = LoraConfig(
                task_type=TaskType.SEQ_CLS,
                r=8,
                lora_alpha=16,
                lora_dropout=0.1,
                bias="none",
                target_modules=[linear_modules[0]]  # Use the first linear module as a test
            )
            model = get_peft_model(base_model, lora_config)
            logger.info(f"Successfully created PEFT model with explicit module: {linear_modules[0]}")

# Print trainable parameters
model.print_trainable_parameters()


NameError: name 'logger' is not defined

In [ ]:
try:
    logger.info("Setting up training arguments")
    training_args = TrainingArguments(
        output_dir=output_dir,
        # Try to use newer parameters first
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        num_train_epochs=EPOCHS,
        learning_rate=2e-4,
        report_to="none",
        logging_dir="./logs"
    )
    
    # Try adding newer parameters if supported
    if hasattr(TrainingArguments, "evaluation_strategy"):
        training_args.evaluation_strategy = "epoch"
        training_args.save_strategy = "epoch"
        training_args.load_best_model_at_end = True
    
    # Try enabling fp16 if supported
    if hasattr(TrainingArguments, "fp16"):
        training_args.fp16 = True
    
except Exception as e:
    logger.error(f"Error setting up TrainingArguments: {e}")
    # Fallback to minimal TrainingArguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        num_train_epochs=EPOCHS,
        learning_rate=2e-4
    )


In [12]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
from sklearn.metrics import accuracy_score, f1_score
import os

# ✅ Load dataset and tokenize
dataset = load_dataset("hendzh/PromptShield")
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

MAX_LEN = 256
BATCH_SIZE = 16
EPOCHS = 3

# Tokenization
def tokenize_fn(example):
    return tokenizer(
        example["prompt"], 
        padding="max_length", 
        truncation=True, 
        max_length=MAX_LEN
    )

tokenized = dataset.map(tokenize_fn, batched=True)
train_dataset = tokenized["train"].with_format("torch")
val_dataset = tokenized["validation"].with_format("torch")

# ✅ Load base model with quantization (4-bit)
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

base_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    quantization_config=bnb_config,
    device_map="auto"
)

# ✅ Prepare model for QLoRA training
base_model = prepare_model_for_kbit_training(base_model)

# Find Linear modules and prepare for LoRA
linear_modules = []
for name, module in base_model.named_modules():
    if isinstance(module, nn.Linear):
        # Capture only intermediate sized linear layers (not tiny ones)
        if module.in_features >= 128:
            linear_modules.append(name)

print(f"Found {len(linear_modules)} linear modules suitable for LoRA")
print("First few modules:", linear_modules[:5])

# ✅ Define QLoRA config with discovered modules
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    # Use the found linear modules
    target_modules=linear_modules
)

# Attempt to create PEFT model
try:
    model = get_peft_model(base_model, lora_config)
    model.print_trainable_parameters()
    print("✅ Successfully created PEFT model!")
except Exception as e:
    print(f"❌ Error creating PEFT model: {e}")
    
    # Fallback to simpler approach
    print("Trying fallback approach...")
    lora_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        r=8,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        # Use regex pattern approach
        target_modules=[".*dense.*"]
    )
    
    try:
        model = get_peft_model(base_model, lora_config)
        model.print_trainable_parameters()
        print("✅ Successfully created PEFT model with fallback approach!")
    except Exception as e2:
        print(f"❌ Fallback also failed: {e2}")
        raise e2

# ✅ TrainingArguments
output_dir = "./qlora-bert-base"
os.makedirs(output_dir, exist_ok=True)

training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    learning_rate=2e-4,
    load_best_model_at_end=True,
    report_to="none",
    logging_dir="./logs",
    fp16=True,
    weight_decay=0.01,
)

# ✅ Compute metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), dim=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='binary')
    return {"accuracy": acc, "f1": f1}

# ✅ Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# ✅ Train the model
trainer.train()

# ✅ Save the LoRA adapter
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model fine-tuned and saved to {output_dir}")

# Optional: Evaluate on test set if available
if "test" in tokenized:
    test_dataset = tokenized["test"].with_format("torch")
    results = trainer.evaluate(test_dataset)
    print(f"Test results: {results}")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\incha\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\incha\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\incha\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\incha\AppData\Roaming\Python\Python310\site-packages\traitlets\config\applicatio

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\incha\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\incha\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\incha\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\incha\AppData\Roaming\Python\Python310\site-packages\traitlets\config\applicatio

AttributeError: _UFUNC_API not found

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
numpy.core.umath failed to import

In [2]:
!pip uninstall torch torchvision torchaudio -y


Found existing installation: torch 2.4.0+cu118
Uninstalling torch-2.4.0+cu118:
  Successfully uninstalled torch-2.4.0+cu118
Found existing installation: torchvision 0.19.0+cu118
Uninstalling torchvision-0.19.0+cu118:
  Successfully uninstalled torchvision-0.19.0+cu118
Found existing installation: torchaudio 2.4.0+cu118
Uninstalling torchaudio-2.4.0+cu118:
  Successfully uninstalled torchaudio-2.4.0+cu118


You can safely remove it manually.


In [4]:
!pip install C:\Users\incha\Downloads\torch-2.1.2+cu118-cp310-cp310-win_amd64.whl


Processing c:\users\incha\downloads\torch-2.1.2+cu118-cp310-cp310-win_amd64.whl

You should consider upgrading via the 'D:\Aventus3\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install -U peft transformers==4.38.2 accelerate datasets bitsandbytes -q


  You can safely remove it manually.


In [7]:
!pip install numpy==1.26.4 --force-reinstall

  Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl (15.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'd:\\aventus3\\venv\\lib\\site-packages\\numpy\\fft\\_pocketfft_umath.cp310-win_amd64.pyd'
Check the permissions.

You should consider upgrading via the 'D:\Aventus3\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [7]:
!pip uninstall transformers -y
!pip install transformers==4.38.2

Found existing installation: transformers 4.38.2
Uninstalling transformers-4.38.2:
  Successfully uninstalled transformers-4.38.2
  Using cached transformers-4.38.2-py3-none-any.whl.metadata (130 kB)
Using cached transformers-4.38.2-py3-none-any.whl (8.5 MB)


In [9]:
!rd /s /q d:\Aventus3\venv\lib\site-packages\transformers

In [10]:
!pip install transformers==4.38.2

In [1]:
!pip install --upgrade --force-reinstall transformers==4.39.3
!pip install --upgrade --force-reinstall peft==0.10.0
!pip install --upgrade --force-reinstall accelerate==0.27.2
!pip install --upgrade --force-reinstall bitsandbytes==0.43.1
!pip install --upgrade --force-reinstall datasets==2.18.0
!pip install --upgrade --force-reinstall tokenizers==0.15.2

  Using cached transformers-4.39.3-py3-none-any.whl.metadata (134 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached huggingface_hub-0.31.2-py3-none-any.whl.metadata (13 kB)
  Using cached numpy-2.2.5-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-win_amd64.whl.metadata (2.1 kB)
  Using cached regex-2024.11.6-cp310-cp310-win_amd64.whl.metadata (41 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tokenizers-0.15.2-cp310-none-win_amd64.whl.metadata (6.8 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
  Using cached charset_no

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.


  Using cached peft-0.10.0-py3-none-any.whl.metadata (13 kB)
  Using cached numpy-2.2.5-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached psutil-7.0.0-cp37-abi3-win_amd64.whl.metadata (23 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-win_amd64.whl.metadata (2.1 kB)
  Using cached torch-2.7.0-cp310-cp310-win_amd64.whl.metadata (29 kB)
  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached accelerate-1.7.0-py3-none-any.whl.metadata (19 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached huggingface_hub-0.31.2-py3-none-any.whl.metadata (13 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.

  You can safely remove it manually.
ERROR: Exception:
Traceback (most recent call last):
  File "D:\Aventus3\venv\lib\site-packages\pip\_internal\cli\base_command.py", line 105, in _run_wrapper
    status = _inner_run()
  File "D:\Aventus3\venv\lib\site-packages\pip\_internal\cli\base_command.py", line 96, in _inner_run
    return self.run(options, args)
  File "D:\Aventus3\venv\lib\site-packages\pip\_internal\cli\req_command.py", line 68, in wrapper
    return func(self, options, args)
  File "D:\Aventus3\venv\lib\site-packages\pip\_internal\commands\install.py", line 459, in run
    installed = install_given_reqs(
  File "D:\Aventus3\venv\lib\site-packages\pip\_internal\req\__init__.py", line 83, in install_given_reqs
    requirement.install(
  File "D:\Aventus3\venv\lib\site-packages\pip\_internal\req\req_install.py", line 867, in install
    install_wheel(
  File "D:\Aventus3\venv\lib\site-packages\pip\_internal\operations\install\wheel.py", line 729, in install_wheel
    _install

  Using cached accelerate-0.27.2-py3-none-any.whl.metadata (18 kB)
  Using cached numpy-2.2.5-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached psutil-7.0.0-cp37-abi3-win_amd64.whl.metadata (23 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-win_amd64.whl.metadata (2.1 kB)
  Using cached torch-2.7.0-cp310-cp310-win_amd64.whl.metadata (29 kB)
  Using cached huggingface_hub-0.31.2-py3-none-any.whl.metadata (13 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl

...

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'd:\\aventus3\\venv\\lib\\site-packages\\numpy-2.2.5.dist-info\\'



  Using cached tokenizers-0.15.2-cp310-none-win_amd64.whl.metadata (6.8 kB)
  Using cached huggingface_hub-0.31.2-py3-none-any.whl.metadata (13 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-win_amd64.whl.metadata (2.1 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
  Using cached charset_normalizer-3.4.2-cp310-cp310-win_amd64.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.4.26-py3-none-any.whl.metadata (2.5 kB)
Using cached tokenizers-0.15.2-cp310-non

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
torchaudio 2.7.0+cu118 requires torch==2.7.0+cu118, but you have torch 2.7.0 which is incompatible.
torchvision 0.22.0+cu118 requires torch==2.7.0+cu118, but you have torch 2.7.0 which is incompatible.


  Using cached bitsandbytes-0.43.1-py3-none-win_amd64.whl.metadata (2.2 kB)
  Using cached torch-2.7.0-cp310-cp310-win_amd64.whl.metadata (29 kB)
  Using cached numpy-2.2.5-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached MarkupSafe-3.0.2-cp310-cp310-win_amd64.whl.metadata (4.1 kB)
Using cached bitsandbytes-0.43.1-py3-none-win_amd64.whl (101.6 MB)
Using cached numpy-2.2.5-cp310-cp310-win_amd64.whl (12.9 MB)
Using cached torch-2.7.0-cp310-cp310-win_amd64.whl (212.5 MB)
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
Using c

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
torchaudio 2.7.0+cu118 requires torch==2.7.0+cu118, but you have torch 2.7.0 which is incompatible.
torchvision 0.22.0+cu118 requires torch==2.7.0+cu118, but you have torch 2.7.0 which is incompatible.


In [1]:
!pip uninstall -y transformers peft accelerate bitsandbytes datasets tokenizers

Found existing installation: transformers 4.39.3
Uninstalling transformers-4.39.3:
  Successfully uninstalled transformers-4.39.3
Found existing installation: tokenizers 0.15.2
Uninstalling tokenizers-0.15.2:
  Successfully uninstalled tokenizers-0.15.2


In [4]:
pip list | grep -E "transformers|peft|accelerate|bitsandbytes|datasets|tokenizers"

Note: you may need to restart the kernel to use updated packages.


'grep' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
!pip install --upgrade --force-reinstall --no-cache-dir transformers==4.39.3
!pip install --upgrade --force-reinstall --no-cache-dir peft==0.10.0
!pip install --upgrade --force-reinstall --no-cache-dir accelerate==0.27.2
!pip install --upgrade --force-reinstall --no-cache-dir bitsandbytes==0.43.1
!pip install --upgrade --force-reinstall --no-cache-dir datasets==2.18.0
!pip install --upgrade --force-reinstall --no-cache-dir tokenizers==0.15.2

   ---------------------------------------- 0.0/8.8 MB ? eta -:--:--
   ---------- ----------------------------- 2.4/8.8 MB 12.2 MB/s eta 0:00:01
   ---------------------- ----------------- 5.0/8.8 MB 11.6 MB/s eta 0:00:01
   -------------------------------- ------- 7.1/8.8 MB 10.9 MB/s eta 0:00:01
   ------------------------------------ --- 8.1/8.8 MB 9.5 MB/s eta 0:00:01
   ---------------------------------------- 8.8/8.8 MB 9.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 11.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ----- ---------------------------------- 1.8/12.9 MB 10.1 MB/s eta 0:00:02
   ------------- -------------------------- 4.5/12.9 MB 11.2 MB/s eta 0:00:01
   --------------------- ------------------ 6.8/12.9 MB 11.3 MB/s eta 0:00:01
   ----------------------------- ---------- 9.4/12.9 MB 11.5 MB/s eta 0:00:01
   -----------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.7.0+cu118 requires torch==2.7.0+cu118, but you have torch 2.7.0 which is incompatible.
torchvision 0.22.0+cu118 requires torch==2.7.0+cu118, but you have torch 2.7.0 which is incompatible.


   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ------ --------------------------------- 2.1/12.9 MB 9.8 MB/s eta 0:00:02
   ---------- ----------------------------- 3.4/12.9 MB 8.7 MB/s eta 0:00:02
   ----------------- ---------------------- 5.5/12.9 MB 8.8 MB/s eta 0:00:01
   --------------------- ------------------ 7.1/12.9 MB 8.6 MB/s eta 0:00:01
   ------------------------- -------------- 8.1/12.9 MB 7.9 MB/s eta 0:00:01
   --------------------------- ------------ 8.9/12.9 MB 7.2 MB/s eta 0:00:01
   ----------------------------- ---------- 9.7/12.9 MB 6.6 MB/s eta 0:00:01
   -------------------------------- ------- 10.5/12.9 MB 6.2 MB/s eta 0:00:01
   --------------------------------- ------ 10.7/12.9 MB 6.0 MB/s eta 0:00:01
   ---------------------------------- ----- 11.3/12.9 MB 5.3 MB/s eta 0:00:01
   ---------------------------------- ----- 11.3/12.9 MB 5.3 MB/s eta 0:00:01
   ------------------------------------- -- 12.1/12.9 MB 4.7 MB/s eta 0:00:01
 

ERROR: Could not install packages due to an OSError: Could not find a suitable TLS CA certificate bundle, invalid path: C:\Program Files\PostgreSQL\16\ssl\certs\ca-bundle.crt

ERROR: Could not install packages due to an OSError: Could not find a suitable TLS CA certificate bundle, invalid path: C:\Program Files\PostgreSQL\16\ssl\certs\ca-bundle.crt

ERROR: Could not install packages due to an OSError: Could not find a suitable TLS CA certificate bundle, invalid path: C:\Program Files\PostgreSQL\16\ssl\certs\ca-bundle.crt

ERROR: Could not install packages due to an OSError: Could not find a suitable TLS CA certificate bundle, invalid path: C:\Program Files\PostgreSQL\16\ssl\certs\ca-bundle.crt

ERROR: Could not install packages due to an OSError: Could not find a suitable TLS CA certificate bundle, invalid path: C:\Program Files\PostgreSQL\16\ssl\certs\ca-bundle.crt



In [1]:
from transformers import AutoTokenizer, Trainer
import torch
import numpy

d:\Aventus3\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
cannot import name 'EncoderDecoderCache' from 'transformers' (d:\Aventus3\venv\lib\site-packages\transformers\__init__.py)

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
from sklearn.metrics import accuracy_score, f1_score
import os
from transformers import BitsAndBytesConfig

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
cannot import name 'is_torch_tpu_available' from 'transformers.utils' (d:\Aventus3\venv\lib\site-packages\transformers\utils\__init__.py)

In [ ]:
dataset = load_dataset("hendzh/PromptShield")
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
MAX_LEN = 256
BATCH_SIZE = 16
EPOCHS = 3

# Tokenize the dataset
def tokenize_fn(example):
    return tokenizer(
        example["prompt"], 
        padding="max_length", 
        truncation=True, 
        max_length=MAX_LEN
    )

tokenized = dataset.map(tokenize_fn, batched=True)
train_dataset = tokenized["train"].with_format("torch")
val_dataset = tokenized["validation"].with_format("torch")

# Configure quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# Load base model with quantization
base_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    quantization_config=bnb_config,
    device_map="auto"
)